# A. Introduction

To ensure data quality, a systematic and automatable data validation process is required. One of flexible open-source tools for data validation is **Great Expectations (GE)**.

In this notebook, cleaned dataset will be validated using 7 expectations. The expected results for all expectation have to **true**. 

# B. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import great_expectations as gx
from great_expectations.data_context import FileDataContext

# C. Dataset Loading

## i. Instantiate Data Context

In [2]:
context = FileDataContext.create(project_root_dir='./')

## ii. Connect to `Datasource`

In [ ]:
# Give a name to a Datasource. This name must be unique between Datasources.
datasource_name = 'csv-datasource_SCM'
datasource = context.sources.add_pandas(datasource_name)

# Give a name to a data asset
asset_name = 'SCMS_delivery_data'
path_to_data = "D:\PORTFOLIO PROJECT\Supply-Chain-Data-Automation-and-Visualization-for-Using-Airflow\data\data_clean.csv"
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Build batch request
batch_request = asset.build_batch_request()

## iii. Create an Expectation Suite

In [6]:
# Creat an expectation suite
expectation_suite_name = 'expectation-SCMS-delivery-dataset'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check the validator
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,unnamed_0,id,project_code,pq,po_so,asn_dn,country,managed_by,fulfill_via,vendor_inco_term,...,unit_of_measure_per_pack,line_item_quantity,line_item_value,pack_price,unit_price,manufacturing_site,first_line_designation,weight_kilograms,freight_cost_usd,line_item_insurance_usd
0,0,1,100-CI-T01,Pre-PQ Process,SCMS-4,ASN-8,Côte d'Ivoire,PMO - US,Direct Drop,EXW,...,30,19,551.0,29.00,0.97,Ranbaxy Fine Chemicals LTD,True,13.0,780.34,0.0
1,1,3,108-VN-T01,Pre-PQ Process,SCMS-13,ASN-85,Vietnam,PMO - US,Direct Drop,EXW,...,240,1000,6200.0,6.20,0.03,"Aurobindo Unit III, India",True,358.0,4521.50,0.0
2,2,4,100-CI-T01,Pre-PQ Process,SCMS-20,ASN-14,Côte d'Ivoire,PMO - US,Direct Drop,FCA,...,100,500,40000.0,80.00,0.80,ABBVIE GmbH & Co.KG Wiesbaden,True,171.0,1653.78,0.0
3,3,15,108-VN-T01,Pre-PQ Process,SCMS-78,ASN-50,Vietnam,PMO - US,Direct Drop,EXW,...,60,31920,127360.8,3.99,0.07,"Ranbaxy, Paonta Shahib, India",True,1855.0,16007.06,0.0
4,4,16,108-VN-T01,Pre-PQ Process,SCMS-81,ASN-55,Vietnam,PMO - US,Direct Drop,EXW,...,60,38000,121600.0,3.20,0.05,"Aurobindo Unit III, India",True,7590.0,45450.08,0.0


# D. Create Expectations

## i. Expectation 1 : To Be Unique

In this expectation, we have to validate ID delivery must contain unique value. Because every ID has different datas of delivery, so it has to be unique.

In [7]:
validator.expect_column_values_to_be_unique('id')

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 10324,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## ii. Expectation 2 : To Be Between Min Value and Max Value

In [8]:
#Expectation 2a : Column `line_item_quantity` must be more than 1 quantity
validator.expect_column_values_to_be_between(
    column='line_item_quantity', min_value=1
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 10324,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

This expectation is to ensure that all items ordered must be minimum 1 pcs.

In [9]:
#Expectation 2b : Column `freight_cost_usd` must be less tah $300,000 per one shipment
validator.expect_column_values_to_be_between(
    column='freight_cost_usd', min_value=0, max_value=300_000
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 10324,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

This expectation is to ensure that freight cost for all delivery is not more than $300.000 USD per one shipment. This value is the maximum budget from the company.

## iii. Expectation 3 : To Be in Set

In [10]:
#Expectation 3a : Column `shipment_mode` must contain specific values

validator.expect_column_values_to_be_in_set(
    'shipment_mode',
    ["Air", "Air Charter", "Truck", "Ocean"]
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 10324,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

This expectation is to ensure that shipment mode will use only 4 modes that company usually use: Air, Air charter, Truck, and Ocean (via Ship)

In [11]:
#Expectation 3b : Column `product_group` must contain specific values

validator.expect_column_values_to_be_in_set(
    'product_group',
    ["ACT", "ANTM", "ARV", "HRDT", "MRDT"]
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 10324,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Since this projects are doing delivery of pharmaceutical products mainly for antivirus and antimalarial, so this expectation is to ensure products that will be delivered belong to one of these product class:
- ACT = Artemisinin-based Combination Therapy (for Malaria)
- ANTM = Antimalaria
- ARV = Antiretroviral
- HRDT = HIV Rapid Diagnostic Test
- MRDT = Malaria Rapid Diagnostic Test

## iv. Expectation 4 : To Be in Type List

In [12]:
# Expectation 4a : Column `first_line_designation` must in form of boolean

validator.expect_column_values_to_be_in_type_list('first_line_designation', ['bool'])

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "bool_"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

This expectation is to ensure that first line designation column is boolean type because it only contain true/false.

In [13]:
# Expectation 4b : Column `unit_price` must in form of float or integer

validator.expect_column_values_to_be_in_type_list('unit_price', ['float', 'int'])

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "float64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

This expectation is to ensure unit price column is float or integer since this column contains price value.

## v. Expectation 5 : Pair Values A To Be Greater Than B

In [14]:
#Expectation 5 : Column `line_item_value` must be more than column `pack_price`
validator.expect_column_pair_values_a_to_be_greater_than_b(
    column_A="line_item_value",
    column_B="pack_price",
    or_equal=True
)

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 10324,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

This expectation is to ensure that column 'line_item_value' must be greater than 'pack_price' because column 'line_item_value' is the total price of the products meanwhile column 'pack_price' is product price per packs.

## vi. Expectation 6 : Most Common Value To Be In Set

In [15]:
#Expectation 6 : Ensure "PMO - US" is the most common value from column `managed_by`
validator.expect_column_most_common_value_to_be_in_set(
    column="managed_by",
    value_set= ["PMO - US"],
    ties_okay=True
)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": [
      "PMO - US"
    ]
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

This expectation is to ensure that all the projects mostly are handled by the company partners which is "PMO - US" or Project Management Office in the USA.

## vii. Expectation 7 : Values To Match Regex

In [16]:
# Expectation 7 : Column `asn_dn` must match a regex pattern by containing "ASN" or "DN"
validator.expect_column_values_to_match_regex(
    column="asn_dn",
    regex=".*(ASN|DN).*"
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 10324,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

This expectation is to ensure that column 'ASN/DN' is contain word ASN or DN because it marked the code of Advance Shipping Notice or Debit Note

# E. Create Checkpoint

In [17]:
# Save into Expectation Suite

validator.save_expectation_suite(discard_failed_expectations=False)

In [18]:
# Create a checkpoint

checkpoint = context.add_or_update_checkpoint(
    name = 'checkpoint_SCMS_delivery_data',
    validator = validator,
)

In [19]:
# Run a checkpoint

checkpoint_result = checkpoint.run()

Calculating Metrics:   0%|          | 0/52 [00:00<?, ?it/s]

In [20]:
# Build data docs

context.build_data_docs()

{'local_site': 'file://d:\\HACKTIV 8 FTDS\\FTDS PHASE 2\\MILESTONE\\p2-ftds027-hck-m3-wandanisrina\\gx\\uncommitted/data_docs/local_site/index.html'}

# F. Conclusion

After cleaned dataset validated using 7 expectations, all the outcomes are **true**. 

This validation can also be used to other similar dataset (that have same columns). Furthermore, another data that will be taken on different period can be validated using this validation since it already saved on the checkpoint.